In [1]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer once
MODEL_PATH = r"D:\ChatBot\phi1.5"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
llm_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, trust_remote_code=True)

# Function to generate AI response
def generate_answer(question, model):
    try:
        prompt = (
            "You are a compassionate and informative AI health assistant.\n"
            "Answer the following health-related question clearly and supportively. Do not include any signature or names at the end:\n\n"

            f"{question.strip()}\n\n"
            "Answer:"
        )

        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        output_ids = model.generate(
            input_ids,
            max_new_tokens=400,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

        # Debug: Show full model output
        raw_output = tokenizer.decode(output_ids[0], skip_special_tokens=False)
        print("=== RAW MODEL OUTPUT ===")
        print(raw_output)
        print("========================")

        # Get only the new tokens after the prompt
        generated_ids = output_ids[0][len(input_ids[0]):]
        answer = tokenizer.decode(generated_ids, skip_special_tokens=True)

        return answer.strip()
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# Gradio interface wrapper
def chatbot(question):
    return generate_answer(question, llm_model)

# Launch the interface
if __name__ == '__main__':
    interface = gr.Interface(
        fn=chatbot,
        inputs=gr.Textbox(
            label="Your Question",
            placeholder="Ask a health-related question...",
            lines=4,
        ),
        outputs=gr.Textbox(
            label="AI Health Assistant Response",
            lines=8,
        ),
        title="🩺 AI Health Assistant",
        description=(
            "Type any general health-related question below and receive helpful guidance from the AI assistant.\n"
            "⚠️ Note: This assistant is for informational purposes only and is **not** a substitute for professional medical advice."
        ),
        theme="soft",
        allow_flagging="never",
        live=True,  # enables typing-effect/streaming-like response
    )
    interface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
